In [1]:
import os, codecs
INPUT_FOLDERS = [
      "../CommonDataModel_Wiki_Files/StandardizedClinicalDataTables"
     ,"../CommonDataModel_Wiki_Files/StandardizedDerivedElements"
     ,"../CommonDataModel_Wiki_Files/StandardizedHealthEconomicsDataTables"
     ,"../CommonDataModel_Wiki_Files/StandardizedHealthSystemDataTables"
     ,"../CommonDataModel_Wiki_Files/StandardizedMetadata"
#      ,"../CommonDataModel_Wiki_Files/StandardizedVocabularies"
]
OUTPUT_FILENAME = "CDMv5.2.0.csv"

In [2]:
# Find all files with table descriptions
filenames = []
for folder in INPUT_FOLDERS:
    filenames.extend([(folder, x) for x in os.listdir(folder)])
filenames.append(("../CommonDataModel_Wiki_Files/StandardizedVocabularies","ATTRIBUTE_DEFINITION.md"))
filenames.append(("../CommonDataModel_Wiki_Files/StandardizedVocabularies","COHORT_DEFINITION.md"))
print([x[1] for x in filenames])

['CONDITION_OCCURRENCE.md', 'DEATH.md', 'DEVICE_EXPOSURE.md', 'DRUG_EXPOSURE.md', 'FACT_RELATIONSHIP.md', 'MEASUREMENT.md', 'NOTE.md', 'NOTE_NLP.md', 'OBSERVATION.md', 'OBSERVATION_PERIOD.md', 'PERSON.md', 'PROCEDURE_OCCURRENCE.md', 'SPECIMEN.md', 'Standardized-Clinical-Data-Tables.md', 'VISIT_DETAIL.md', 'VISIT_OCCURRENCE.md', 'COHORT.md', 'COHORT_ATTRIBUTE.md', 'CONDITION_ERA.md', 'DOSE_ERA.md', 'DRUG_ERA.md', 'Standardized-Derived-Elements.md', 'COST.md', 'PAYER_PLAN_PERIOD.md', 'Standardized-Health-Economics-Data-Tables.md', 'CARE_SITE.md', 'LOCATION.md', 'PROVIDER.md', 'Standardized-Health-System-Data-Tables.md', 'CDM_SOURCE.md', 'Standardized-Metadata.md', 'ATTRIBUTE_DEFINITION.md', 'COHORT_DEFINITION.md']


In [5]:
# Parse all field description tables into one table
counter = 0
tables_processed = set()
with codecs.open(OUTPUT_FILENAME,'w','utf-8') as file_out:
    file_out.write("TABLE_NAME,COLUMN_NAME,IS_NULLABLE,DATA_TYPE,DESCRIPTION")
    file_out.write("\n")
    for folder, filename in filenames:
        is_header_seen = False
        with codecs.open(os.path.join(folder, filename), 'r', 'windows-1252') as file_in:
            for line in file_in:
                # Table rows contain a pipe
                if '|' in line:
                    # Header rows are formatted with :-
                    # Only first header taken into account
                    if ':-' in line:
                        if is_header_seen:
                            break;
                        is_header_seen = True
                        continue

                    if is_header_seen:
                        line = line.strip().strip('|')
                        columns = line.split('|')

                        # Field description table contains 4 columns.
                        if len(columns) != 4:
                            continue
                        
                        field_name = columns[0].strip().upper()
                        required = columns[1].strip().lower()
                        field_type = columns[2].strip().lower()
                        description = '"' + columns[3].strip().replace('"','') + '"' 
                        
                        if field_type == "no" or field_type == "yes":
                            print("Switching columns 'field type' and 'required' for %-20s %s" % (filename, columns[0]))
                            #switcheroo
                            temp = field_type
                            field_type = required
                            required = temp

                        # is_nullable = !required
                        is_nullable = "NO" if required == "yes" else "YES"

                        # convert field types
                        if "varchar" in field_type or "string" in field_type:
                            #varchar(xxx)
                            data_type = "CHARACTER VARYING"
                        elif "lob" in field_type:
                            #clob or blob
                            data_type = "TEXT"
                        elif field_type == "float":
                            data_type = "NUMERIC"
                        else:
                            data_type = field_type.upper()                    
                        
                        #Table name from file name
                        table_name = filename.replace('.md','')
                        row = ",".join((table_name, field_name, is_nullable, data_type, description))
                        #print(row)
                        file_out.write(row)
                        file_out.write("\n")
                        tables_processed.add(filename)
                        counter += 1
print()
print("%d rows processed from %d tables written to %s" % (counter, len(tables_processed),OUTPUT_FILENAME))
a = [print(x) for x in sorted(tables_processed)]

Switching columns 'field type' and 'required' for VISIT_DETAIL.md      admitting_source_value	
Switching columns 'field type' and 'required' for VISIT_DETAIL.md      discharge_to_source_value
Switching columns 'field type' and 'required' for VISIT_DETAIL.md      discharge_to_concept_id
Switching columns 'field type' and 'required' for VISIT_DETAIL.md      preceding_visit_detail_id	
Switching columns 'field type' and 'required' for VISIT_DETAIL.md      visit_detail_parent_id	
Switching columns 'field type' and 'required' for VISIT_DETAIL.md      visit_occurrence_id	
Switching columns 'field type' and 'required' for VISIT_OCCURRENCE.md  admitting_source_value	
Switching columns 'field type' and 'required' for VISIT_OCCURRENCE.md  discharge_to_concept_id
Switching columns 'field type' and 'required' for VISIT_OCCURRENCE.md  discharge_to_source_value
Switching columns 'field type' and 'required' for VISIT_OCCURRENCE.md  preceding_visit_occurrence_id	
Switching columns 'field type' and 'req